# 1. Create a Database from a csv file using Milvus and Pandas
- Read cleaned data
- Create a local database for testing
- Add necessary data to database

## Mimic [milvus_quickstart](./milvus_quickstart.ipynb) first and then do explorations

In [1]:
import pandas as pd
from pymilvus import model
from pymilvus import connections, utility, FieldSchema, CollectionSchema, DataType, Collection

## 1.1. Read Data

In [25]:
df = pd .read_csv("/workspaces/Music_Playlist_Generation/music_playlist_generation/data/data.csv")
df.head()

,danceability,track_genre,valence,track_name
0,0.676,acoustic,0.715,Comedy
1,0.420,acoustic,0.267,Ghost - Acoustic
2,0.438,acoustic,0.120,To Begin Again
3,0.266,acoustic,0.143,Can't Help Falling In Love
4,0.618,acoustic,0.167,Hold On


Here track_name is supposed to be our output data from Vector DB given other factors.

## 1.2 Setting up Milvus DB

In [4]:
try:
    conn = connections.connect("default", host="localhost", port="19530")
    print("Connected to Milvus.")
except Exception as e:
    print(f"Failed to connect to Milvus: {e}")
    raise

Connected to Milvus.


### 1.2.1 Creating a Collection

A collection in Milvus is like a table in a traditional database. It's where our data will be stored. Each collection can have multiple fields, akin to columns in a table. A collection a `primary_key` field which is a unique identifier for each entity within a collection. It ensures that each entity can be uniquely identified and accessed.

In [26]:
if utility.has_collection("music_collection"):
    print("Deleting old collection")
    utility.drop_collection("music_collection")

# Define fields for our collection
fields = [
    FieldSchema(name="music_id", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=100),
    FieldSchema(name="track_genre", dtype=DataType.VARCHAR, max_length=500),
    # FieldSchema(name="danceability", dtype=DataType.FLOAT),
    # FieldSchema(name="valence", dtype=DataType.FLOAT),
    FieldSchema(name="danceability_valence", dtype=DataType.FLOAT_VECTOR, dim=2),
    FieldSchema(name="track_name", dtype=DataType.VARCHAR, max_length=500),
    # FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=384)
]

schema = CollectionSchema(fields, description="Collection of Music")
collection = Collection("music_collection", schema)

Deleting old collection


### 1.2.2 Data Preparation

- Convert Dataframe to list of dict (each dict is a new row) or alternative `Collection.construct_from_dataframe`
- Vectorize data using an embedding model. Vectorized data will be used for searching through the database so in this scenario we vectorize the combination of `danceability`, `track_genre` and `valence`.

In [28]:
# This will download a small embedding model "paraphrase-albert-small-v2" (~50MB).
# embedding_fn = model.dense.SentenceTransformerEmbeddingFunction(
#     model_name='multi-qa-MiniLM-L6-cos-v1', # Specify the model name
#     device='cuda:0' # Specify the device to use, e.g., 'cpu' or 'cuda:0'
# )

# Convert data to list
# Limiting data to 50 values due to limit on local DB
track_genre = df["track_genre"].to_list()[:50]
# danceability = df["danceability"].to_list()[:50]
# valence = df["valence"].to_list()[:50]
danceability_valence = list(zip(df["danceability"].to_list()[:50], df["valence"].to_list()[:50]))
track_name = df["track_name"].to_list()[:50]
# embeddings = embedding_fn.encode_documents(track_genre)

### 1.2.3 Adding Data to DB

In [30]:
data = [
    {
        "music_id": str(i),
        # "embeddings": embeddings[i],
        "track_genre": track_genre[i],
        # "danceability": danceability[i],
        # "valence": valence[i],
        "danceability_valence": danceability_valence[i],
        "track_name": track_name[i],
    }
    for i in range(len(danceability_valence))
]

# index_params = {
#   "metric_type":"L2",
#   "index_type":"IVF_FLAT",
#   "params":{"nlist":128}
# }
# collection.create_index("embeddings", index_params)
insert_result = collection.insert(data)
print(insert_result)

(insert count: 50, delete count: 0, upsert count: 0, timestamp: 452725333411758083, success count: 50, err count: 0


## 1.3 Semantic Search
Get `track_name` by giving values of `danceability`, `track_genre` and `valence` in the format as in input data of dataframe.

danceability: Danceability measures how suitable a track is for dancing, ranging from 0 to 1. Tracks with high danceability scores are
more energetic and rhythmic, making them ideal for dancing.

track_genre: The genre of the track. Due to limiting data to 50 values the default of track_genre is acoustic.

valence: Valence measures the musical positiveness conveyed by a track, ranging from 0 to 1. High valence values indicate more positive
or happy tracks, while lower values suggest more negative or sad ones.

In [31]:
query_template = lambda danceability, track_genre, valence: f"danceability:{danceability} track_genre:{track_genre} valence: {valence}"

# danceability, Max: 0.796, Min: 0.266
# valence, Max: 0.754, Min: 0.0765
########################################
# Max dancebility: 24         0.796    acoustic    0.754   Unlonely
# Min dancebility: 3         0.266    acoustic    0.143  Can't Help Falling In Love
# Max valence: 24         0.796    acoustic    0.754   Unlonely
# Min valence:  6          0.407    acoustic   0.0765  Say Something
check_vals = [
    [1, "acoustic", 1],
    [0, "acoustic", 0],
    [0.5, "acoustic", 0.5],
    [1, "acoustic", 0],
    [0, "acoustic", 1],
    [1, "acoustic", 0.5],
    [0.5, "acoustic", 1],
]

collection.load()
sens = 0.5
for idx, val in enumerate(check_vals):
    x_val, genre, y_val = val
    # query_vector = embedding_fn.encode_queries([genre])
    result = collection.search(
        data=[[x_val, y_val]],  # query vectors
        anns_field="danceability_valence",
        param={"metric_type": "L2", "params": {"nprobe": 10}},
        limit=5,  # number of returned entities
        expr=f"track_genre=={genre}",
        output_fields=["valence", "danceability"],  # specifies fields to be returned
    )[0]
    # print(idx, list(map(lambda x: x.id, result)))
    print(idx, result)

RPC error: [load_collection], <MilvusException: (code=700, message=index not found[collection=music_collection])>, <Time:{'RPC start': '2024-09-22 13:03:13.401564', 'RPC error': '2024-09-22 13:03:13.403599'}>


MilvusException: <MilvusException: (code=700, message=index not found[collection=music_collection])>

In [119]:
result

["id: 32, distance: 3.037962676560868e-13, entity: {'valence': 0.5640000104904175, 'danceability': 0.5929999947547913}", "id: 37, distance: 3.037962676560868e-13, entity: {'valence': 0.3160000145435333, 'danceability': 0.5009999871253967}", "id: 38, distance: 3.037962676560868e-13, entity: {'valence': 0.414000004529953, 'danceability': 0.6060000061988831}", "id: 41, distance: 3.037962676560868e-13, entity: {'valence': 0.1420000046491623, 'danceability': 0.5680000185966492}", "id: 42, distance: 3.037962676560868e-13, entity: {'valence': 0.7250000238418579, 'danceability': 0.5680000185966492}"]

In [120]:
# Calculate combined scores (adjust weights as needed)
tracks = {}

for entity in result:
    # Calculate scores for each criteria (e.g., cosine similarity for track_genre, Euclidean distance for valence and danceability)
    genre_score = entity.entity.distance
    valence_score = abs(entity.entity.valence - x_val)  # Adjust for closeness to 1
    danceability_score = abs(entity.entity.danceability - y_val)  # Adjust for closeness to 1

    tracks[entity.entity.id] = genre_score + valence_score + danceability_score

# # Rank results based on combined score
search_results = dict(sorted(tracks.items(), key=lambda item: item[1]))

print(search_results)
# print(tracks)

{'32': 0.47100001573593003, '38': 0.47999998927146775, '42': 0.6570000052455125, '37': 0.6829999983313737, '41': 0.7899999767544923}
